In [19]:
import spacy
import json
import glob
import os
from tqdm import tqdm
import torch
import pandas as pd
import scipy.stats as stats

In [5]:
data = {}
for folder in glob.glob("logs/*"):
    overall = f"{folder}/overall/overall.json"
    if not os.path.exists(overall): continue
    with open(overall, 'r') as f:
        cur = json.load(f)
        data[folder[len("logs/"):]] = cur

In [131]:
rows = []
order = set()
for setting in data:
    for model in data[setting]:

        # get param nums
        try:
            with open(f'logs/plain_sampling/{model.replace("/", "-")}.json', 'r') as f:
                params = json.load(f)["metadata"]["num_parameters"]
                order.add((params, model))
        except:
            pass

        # get data
        for sent in data[setting][model]:
            for generations in data[setting][model][sent]['results']:
                token = generations['text'][len(sent):].split(" ")
                resolution = None
                if len(token) == 1:
                    token = ""
                else:
                    token = token[1].strip(",.:?! '\"").replace("\n", "\\n").replace(" ", "_")
                    for option in data[setting][model][sent]['counts']:
                        if generations['resolved'].split(". ")[1].startswith(option):
                            resolution = option
                            break
                rows.append({
                    "setting": setting,
                    "model": model.split("/")[-1].split(".json")[0] if "logs/" in model else model.replace("/", "-"),
                    "sent": sent,
                    "token": token,
                    "resolution": resolution
                })

df = pd.DataFrame(rows)

order = sorted(order, key=lambda x: x[0])
order = [x[1].replace("/", "-") for x in order]
df['model'] = pd.Categorical(df['model'].astype(str))
df['model'].cat.set_categories(order, inplace=True)

df

/var/folders/ks/tsqyvj_n5bv55g97m0fc7tb40000gn/T/ipykernel_6557/976839220.py:40: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.


,setting,model,sent,token,resolution
0,top_p_0.5,EleutherAI-pythia-2.8b,John seized the comic from Bill. He,read,John
1,top_p_0.5,EleutherAI-pythia-2.8b,John seized the comic from Bill. He,read,John
2,top_p_0.5,EleutherAI-pythia-2.8b,John seized the comic from Bill. He,gave,John
3,top_p_0.5,EleutherAI-pythia-2.8b,John seized the comic from Bill. He,said,John
4,top_p_0.5,EleutherAI-pythia-2.8b,John seized the comic from Bill. He,said,John
...,...,...,...,...,...
62395,top_k_50,EleutherAI-pythia-1b,Brittany was amazed by Amanda. She,was,Brittany
62396,top_k_50,EleutherAI-pythia-1b,Brittany was amazed by Amanda. She,would,Brittany
62397,top_k_50,EleutherAI-pythia-1b,Brittany was amazed by Amanda. She,had,Amanda
62398,top_k_50,EleutherAI-pythia-1b,Brittany was amazed by Amanda. She,had,Brittany


In [132]:
for sent in df['sent'].unique():
    print(sent)

John seized the comic from Bill. He
John passed the comic to Bill. He
Joseph hit Patrick. He
Patrick was hit by Joseph. He
Ken admired Geoff. He
Ken impressed Geoff. He
Simon ran towards Richard. He
Simon ran away from Richard. He
John passed a comic to Bill. He
John was passing a comic to Bill. He
Amanda amazed Brittany. She
Brittany was amazed by Amanda. She


In [155]:
%config InlineBackend.figure_formats = ['svg']

from plotnine import ggplot, aes, facet_wrap, facet_grid, geom_bar, theme, element_text, geom_errorbar, ggtitle, geom_hline, geom_point
from plotnine.scales import scale_color_manual, scale_x_log10, ylim

sent = 'Brittany was amazed by Amanda. She'
filtered = df[df['sent'] == sent]
filtered = filtered[filtered['setting'] == 'plain_sampling']
filtered = filtered.dropna()

# count up by token column and set token order by count
filtered = filtered.groupby(['token', 'model', 'resolution', 'sent', 'setting']).size().reset_index(name='count')

# print(len(df))
plot = (ggplot(filtered, aes(x='token', y='count', fill='resolution'))
        + geom_bar(stat='identity')
        + ggtitle(sent)
        + theme(axis_text_x=element_text(rotation=90, hjust=0.5, size=5), figure_size=(10, 15))
        + facet_wrap("model", ncol=1)
        )
plot
# plot

<ggplot: (806837589)>

In [162]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

In [227]:
sents = ['John passed a comic to Amanda. He', 'John passed a comic to Amanda. She', 'John passed a comic to Bill. He']
inputs = tokenizer(sents, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss, logits = outputs[:2]

# convert logits to probs
print(logits.shape)

topks = set()
for i in range(logits.shape[0]):
    topk = torch.topk(logits[i, -1, :], 20)
    for tok in topk.indices:
        topks.add(tok.item())

topks = list(topks)
topks.sort(key=lambda x: logits[0, -1].softmax(0)[x].item() - logits[1, -1].softmax(0)[x].item())

print(sents)
print(f"{'':<20}", end="")
for i in range(logits.shape[0]):
    print(f"{sents[i].split('. ')[1]:>10}", end="")
print()
for tok in topks:
    print(f"{tokenizer.decode(tok).replace(' ', '_'):<20}", end="")
    for i in range(logits.shape[0]):
        print(f"{logits[i, -1].softmax(0)[tok].item():>10.2%}", end="")
    print(f"{logits[1, -1].softmax(0)[tok].item() - logits[0, -1].softmax(0)[tok].item():>10.2%}", end="")
    print()

torch.Size([3, 8, 50257])
['John passed a comic to Amanda. He', 'John passed a comic to Amanda. She', 'John passed a comic to Bill. He']
                            He       She        He
_read                    0.90%     6.00%     2.28%     5.10%
_laughed                 0.49%     4.16%     1.86%     3.66%
_looked                  1.16%     3.04%     1.95%     1.88%
_liked                   0.42%     1.86%     1.12%     1.44%
_loved                   0.34%     1.46%     0.90%     1.13%
_took                    1.12%     2.08%     1.28%     0.95%
_drew                    0.55%     1.49%     0.66%     0.94%
_smiled                  0.36%     1.24%     0.39%     0.88%
_wrote                   1.63%     2.47%     2.27%     0.85%
_thought                 1.19%     1.83%     2.08%     0.64%
_was                     7.91%     8.51%     9.10%     0.59%
_got                     0.69%     1.00%     1.02%     0.31%
_didn                    1.58%     1.84%     2.14%     0.25%
_saw               

In [226]:
sents = ['Amanda amazed John. He', 'Amanda amazed John. She', 'Amanda amazed Brittany. She']
inputs = tokenizer(sents, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss, logits = outputs[:2]

# convert logits to probs
print(logits.shape)

topks = set()
for i in range(logits.shape[0]):
    topk = torch.topk(logits[i, -1, :], 20)
    for tok in topk.indices:
        topks.add(tok.item())

topks = list(topks)
topks.sort(key=lambda x: logits[0, -1].softmax(0)[x].item() - logits[1, -1].softmax(0)[x].item())

print(sents)
print(f"{'':<20}", end="")
for i in range(logits.shape[0]):
    print(f"{sents[i].split('. ')[1]:>10}", end="")
print()
for tok in topks:
    print(f"{tokenizer.decode(tok).replace(' ', '_'):<20}", end="")
    for i in range(logits.shape[0]):
        print(f"{logits[i, -1].softmax(0)[tok].item():>10.2%}", end="")
    print(f"{logits[1, -1].softmax(0)[tok].item() - logits[0, -1].softmax(0)[tok].item():>10.2%}", end="")
    print()

torch.Size([3, 6, 50257])
['Amanda amazed John. He', 'Amanda amazed John. She', 'Amanda amazed Brittany. She']
                            He       She       She
_had                     8.09%    10.33%     9.42%     2.24%
_said                    6.87%     8.12%     7.81%     1.25%
_knew                    2.08%     3.06%     2.87%     0.98%
_was                    12.71%    13.56%    12.06%     0.86%
_told                    1.32%     1.73%     2.12%     0.41%
_could                   1.51%     1.89%     1.78%     0.38%
_wanted                  0.35%     0.60%     0.71%     0.26%
_seemed                  0.85%     1.02%     0.57%     0.18%
's                       0.53%     0.70%     1.04%     0.17%
_saw                     0.57%     0.65%     0.71%     0.07%
_did                     0.79%     0.77%     0.57%    -0.02%
_felt                    0.90%     0.86%     1.03%    -0.04%
_wasn                    1.19%     1.15%     1.01%    -0.04%
_took                    0.77%     0.65%     

In [223]:
sents = ['John was hit by Amanda. He was', 'John was hit by Amanda. She was', 'John was hit by Bill. He was']
inputs = tokenizer(sents, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss, logits = outputs[:2]

# convert logits to probs
print(logits.shape)

topks = set()
for i in range(logits.shape[0]):
    topk = torch.topk(logits[i, -1, :], 20)
    for tok in topk.indices:
        topks.add(tok.item())

topks = list(topks)
topks.sort(key=lambda x: logits[0, -1].softmax(0)[x].item() - logits[1, -1].softmax(0)[x].item())

print(sents)
print(f"{'':<20}", end="")
for i in range(logits.shape[0]):
    print(f"{sents[i].split('. ')[1]:>10}", end="")
print()
for tok in topks:
    print(f"{tokenizer.decode(tok).replace(' ', '_'):<20}", end="")
    for i in range(logits.shape[0]):
        print(f"{logits[i, -1].softmax(0)[tok].item():>10.2%}", end="")
    print(f"{logits[1, -1].softmax(0)[tok].item() - logits[0, -1].softmax(0)[tok].item():>10.2%}", end="")
    print()

torch.Size([3, 8, 50257])
['John was hit by Amanda. He was', 'John was hit by Amanda. She was', 'John was hit by Bill. He was']
                        He was   She was    He was
_driving                 0.38%     2.93%     0.76%     2.55%
_a                       1.47%     3.98%     2.67%     2.51%
_trying                  1.05%     2.90%     1.37%     1.85%
_running                 0.41%     1.50%     0.88%     1.09%
_hitting                 0.19%     1.13%     0.91%     0.94%
_on                      1.53%     2.41%     2.51%     0.88%
_standing                0.52%     1.36%     1.01%     0.84%
_going                   0.68%     1.47%     1.06%     0.80%
_the                     0.66%     1.42%     0.91%     0.76%
_wearing                 0.62%     1.37%     0.62%     0.75%
_walking                 0.75%     1.32%     1.09%     0.57%
_just                    1.01%     1.34%     1.70%     0.33%
_so                      1.15%     1.43%     1.37%     0.28%
_screaming               0.8

In [167]:
sent = 'John passed the comic to Amanda. She'
inputs = tokenizer(sent, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss, logits = outputs[:2]

# print top 20 tokens and convert to text, pretty print with probability
topk = torch.topk(logits[0, -1, :], 20)
topk_tokens = [tokenizer.decode([token]) for token in topk.indices]
topk_probs = topk.values.softmax(0).tolist()
for token, prob in zip(topk_tokens, topk_probs):
    print(f"{token.replace(' ', '_'):<20} {prob:.5f}")

_was                 0.17728
_read                0.09698
_wrote               0.07843
_had                 0.07282
_said                0.06710
_asked               0.05948
_told                0.05252
_took                0.04851
_loved               0.04335
_liked               0.03716
_looked              0.03353
_thought             0.03270
_gave                0.03247
_didn                0.03186
_sent                0.03014
_immediately         0.02478
_then                0.02342
_laughed             0.02112
's                   0.01872
_agreed              0.01764


In [168]:
sent = 'John passed the comic to Bill. He'
inputs = tokenizer(sent, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss, logits = outputs[:2]

# print top 20 tokens and convert to text, pretty print with probability
topk = torch.topk(logits[0, -1, :], 20)
topk_tokens = [tokenizer.decode([token]) for token in topk.indices]
topk_probs = topk.values.softmax(0).tolist()
for token, prob in zip(topk_tokens, topk_probs):
    print(f"{token.replace(' ', '_'):<20} {prob:.5f}")

_said                0.17432
_was                 0.17032
_wrote               0.07279
_told                0.06880
_had                 0.06721
_asked               0.06346
_read                0.04212
_didn                0.04210
_thought             0.04020
_loved               0.03038
_liked               0.02884
_took                0.02778
_sent                0.02728
_gave                0.02508
's                   0.02232
_then                0.02196
_looked              0.02187
'd                   0.01895
_did                 0.01723
_got                 0.01700
